In [ ]:
import tkinter
from statistics import mean
%run DataProcessing.ipynb
%run HyperParameterTuning.ipynb
%run Statistics.ipynb
%run visualization.ipynb

In [1]:
class interface:
    def __init__(self):
        
        
        root = tkinter.Tk()
        root.title('Trauma Triage Pipeline')
        root.geometry("640x475")
        root.configure(bg = '#23272a')
        
        self.display = tkinter.Label(text = "", bg = '#99aab5', fg = '#23272a', width = 51, height = 31, wraplength = 350, justify = 'left')
        self.display.place(x = 275, y = 2)
        
        tune_hp_button = tkinter.Button(root, text = "Tune Hyper Parameters", bg = '#99aab5', fg = '#23272a', width = 37, command = lambda: self.tune_action())
        tune_hp_button.place(x = 3, y = 3)
        stats_button = tkinter.Button(root, text = "Process Stats", bg = '#99aab5', fg = '#23272a', width = 37, command = lambda: self.stats_action())
        stats_button.place(x = 3, y = 30)
        roc_plt_button = tkinter.Button(root, text = "Generate ROC Plot", bg = '#99aab5', fg = '#23272a', width = 37, command = lambda:self.roc_plt_action())
        roc_plt_button.place(x = 3, y = 57)
        feature_sig_button = tkinter.Button(root, text = "Generate Feature Sig Plot", bg = '#99aab5', fg = '#23272a', width = 37, command = lambda:self.feat_sig_action())
        feature_sig_button.place(x = 3, y = 84)
        tree_plt_button = tkinter.Button(root, text = "Generate Decision Tree Plot", bg = '#99aab5', fg = '#23272a', width = 37, command = lambda:self.tree_plt_action())
        tree_plt_button.place(x = 3, y = 111)
#         self.display.grid_propagate(0)
        
#         self.display.configure(text = "ALL TEXT HERE")
        
        root.mainloop()
        
        
    def tune_action(self):
        self.dp = DataProcess('fullEDfullmodel.csv')
        X, y = self.dp.get_xy() 
        tune = HyperParameterTuning('hyper_parameter')
        self.prefix = tune.get_prefix()
        
        self.clfs, self.params, self.sen_arr, self.spec_arr, self.thresh_arr, self.cf_matrix_arr = tune.gridSearchCustom(self.dp.get_paramgrid(), X, y)
        self.best_clf, self.best_params, self.best_sen, self.best_spec, self.best_thresh, self.best_conf_matrix, self.y_data = tune.get_best()
        
        tune_display = f'File created: hyper_parameter/{self.prefix}-hyperparameter.csv\n\nTakes: Parameter Grid, X, y\nOutputs: clf, params, sen, spec, thresh, conf_matrix\n\nBest Parameters: {self.best_params}\nBest Sensitivity: {mean(self.best_sen)}\nBest Specificity: {mean(self.best_spec)}\nConfusion Matrix: {self.best_conf_matrix[0]}'
        
        self.display.configure(text = tune_display)
        
    def stats_action(self):
        try:
            stats = Statistics('statistics', self.prefix)
            stats.get_stats(self.spec_arr, self.params)
            self.display.configure(text = f'File Created: statistics/{self.prefix}-statistics.csv\n\nTakes: Specificty List, Prameter pairings\nOutput: Mean of Specificty, Variance of Specificity, Standard Deviation of Specificity\n\n{stats.get_last()}')
        
        except:
            print("No Hyper Parameter Tuning on stack")
        
    def roc_plt_action(self):
        try:
            vis = Visualize('figures', self.prefix)
            vis.roc_curve_plt(self.y_data[0], self.y_data[1])
            self.display.configure(text = f'Generated ROC Plot as: figures/{self.prefix}-roc.png')
        except:
            print("No Hyper Parameter Tuning on stack")
            
    def feat_sig_action(self):
        try:
            vis = Visualize('figures', self.prefix)
            vis.feature_sig_plt(self.best_clf, dp.get_ds(), 10)
            self.display.configure(text = f'Generated Feature Signicance Plot as: figures/{self.prefix}-feat-sig.png')
        except:
            print("No Hyper Parameter Tuning on stack")
            
    def tree_plt_action(self):
        try:
            vis = Visualize('figures', self.prefix)
            vis.tree_plt(self.best_clf, self.dp.get_ds())
            self.display.configure(text = f'Generated Decision Tree Plot as: figures/{self.prefix}-tree.png')
        except:
            print("No Hyper Parameter Tuning on stack")
    
#     def run_pipeline(self,):
        
        